In [7]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import TweetTokenizer
from transform import TextPreprocessor


# Cargar los datos en un DataFrame de pandas
movies_df = pd.read_csv('./data/MovieReviews.csv', sep=',')

# Función para tokenizar los tweets
nltk.download('punkt')
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

# Descargando las stopwords
nltk.download('stopwords')
stop_words_complete = list(stopwords.words('spanish')) + list(stopwords.words('english'))


# Crear el pipeline
text_pipeline = Pipeline([
    ('preprocessing', TextPreprocessor() ),
    ('vectorizer', CountVectorizer(tokenizer=tokenizer, stop_words=stop_words_complete, lowercase=True)),
    ('classifier', RandomForestClassifier(random_state=2))
])


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(movies_df['review_es'], movies_df['sentimiento'], test_size=0.2, random_state=42)

# Ajustar el pipeline a los datos de entrenamiento
text_pipeline.fit(X_train, y_train)

# Evaluar el pipeline en los datos de prueba
score = text_pipeline.score(X_test, y_test)
print("Accuracy:", score)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Esteban\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Esteban\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


IndexError: tuple index out of range